In [58]:
# Locate to working directory
%cd '/home/abaric/retriever-sentiment/'
%load_ext autoreload
%autoreload 2

/home/abaric/retriever-sentiment
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
# Local directory
%cd '/home/abaric/TakeLab/projects/Retriever/Sentiment/retriever-sentiment/'

[Errno 2] No such file or directory: '/home/abaric/TakeLab/projects/Retriever/Sentiment/retriever-sentiment/'
/home/abaric/retriever-sentiment


In [59]:
# Server directory
%cd /home/abaric/retriever-sentiment/

/home/abaric/retriever-sentiment


In [60]:
import pandas as pd
import numpy as np
import re

# Gold-label aggregation

## Import data

In [61]:
# Import labeled non-aggregated data
labeled_df = pd.read_csv('data/labeled/labeled_3k.csv')
labeled_df

,document_id,text,label,annotator,GL,Revision,Comment
0,39899,Glavni pregovarač EU-a za <strong>Bexit</stron...,Error,kostanjeveckinikol,False,False,NaN
1,39899,Glavni pregovarač EU-a za <strong>Bexit</stron...,Sentiment - NEUTR,Anamarija.petrekovic,False,False,NaN
2,39899,Glavni pregovarač EU-a za <strong>Bexit</stron...,Sentiment - NEUTR,SZ,False,False,NaN
3,39899,Glavni pregovarač EU-a za <strong>Bexit</stron...,Sentiment - NEUTR,bdezan,False,False,NaN
4,39899,Glavni pregovarač EU-a za <strong>Bexit</stron...,Sentiment - NEUTR,jlipovac,False,False,NaN
...,...,...,...,...,...,...,...
35672,80805,Zagrebački ‘plinari‘ nastavili negativni niz u...,Ton - NEG,dostric,False,False,NaN
35673,80805,Zagrebački ‘plinari‘ nastavili negativni niz u...,Ton - NEG,jlipovac,False,False,NaN
35674,80805,Zagrebački ‘plinari‘ nastavili negativni niz u...,Ton - NEG,josmanovic,False,False,NaN
35675,80805,Zagrebački ‘plinari‘ nastavili negativni niz u...,Ton - NEG,kostanjeveckinikol,False,False,NaN


## Detect Error labels

In [62]:
# Get ids od documents that contain at least one Error label
error_idx = labeled_df[labeled_df['label'] == 'Error']['document_id']

# Get df with error data
error_df = labeled_df[labeled_df['document_id'].isin(error_idx)]
error_df


,document_id,text,label,annotator,GL,Revision,Comment
0,39899,Glavni pregovarač EU-a za <strong>Bexit</stron...,Error,kostanjeveckinikol,False,False,NaN
1,39899,Glavni pregovarač EU-a za <strong>Bexit</stron...,Sentiment - NEUTR,Anamarija.petrekovic,False,False,NaN
2,39899,Glavni pregovarač EU-a za <strong>Bexit</stron...,Sentiment - NEUTR,SZ,False,False,NaN
3,39899,Glavni pregovarač EU-a za <strong>Bexit</stron...,Sentiment - NEUTR,bdezan,False,False,NaN
4,39899,Glavni pregovarač EU-a za <strong>Bexit</stron...,Sentiment - NEUTR,jlipovac,False,False,NaN
...,...,...,...,...,...,...,...
35000,18774,LEGENDA HNL-A NA KLUPI LOKOMOTIVE: U klubu s K...,Error,vmilic,False,False,3
35001,18774,LEGENDA HNL-A NA KLUPI LOKOMOTIVE: U klubu s K...,Sentiment - NEUTR,LPetesic,False,False,NaN
35002,18774,LEGENDA HNL-A NA KLUPI LOKOMOTIVE: U klubu s K...,Sentiment - NEUTR,josmanovic,False,False,NaN
35003,18774,LEGENDA HNL-A NA KLUPI LOKOMOTIVE: U klubu s K...,Ton - NEG,josmanovic,False,False,NaN


In [63]:
only_error = error_df[error_df['label'] == 'Error']
pd.DataFrame(only_error.groupby('text').size().sort_values().reset_index())

,text,0
0,"""<strong>Ostavljam te</strong> "" na BOK FESTU ...",1
1,Blanka Vlašić u velikom stilu zatvorila najusp...,1
2,Blizanci <strong>Winklevoss</strong> postali p...,1
3,Bobić uprskao dobre <strong>Vijesti</strong> b...,1
4,"DOBROTVORNI KONCERT ""<strong>MI SMO NJIHOV GLA...",1
...,...,...
115,"Ministar Božinović, sisački župan Ivo Žinić i ...",6
116,POČELA OPERACIJA SPAŠAVANJA BRANITELJA MARIUPO...,6
117,POSTAT ĆE NAJPLAĆENIJI U ENGLESKOJ: Ali njemu ...,6
118,<strong>Tanka</strong> linija na festivalu u M...,6


In [64]:
# Get number of documents that contain at least one Error label
len(error_df['document_id'].unique())

120

In [65]:
# Drop documents that contain at least one error label
labeled_df = labeled_df[~(labeled_df['document_id'].isin(error_idx))]
len(labeled_df['document_id'].unique())

2880

## Aggregate labels

In [66]:
# Function for calculating label distribution per category and label aggregation
def get_label_distribution(df, all_labels, sentiment_labels, tone_labels):

    # Init aggregated labels dataframe
    aggregated_labels = pd.DataFrame(columns=['document_id', 
                                        'text', 
                                        'Sentiment - NEG', 
                                        'Sentiment - NEUTR', 
                                        'Sentiment - POZ', 
                                        'aggregated_sentiment', 
                                        'Ton - NEG', 
                                        'Ton - NEUTR', 
                                        'Ton - POZ',
                                        'aggregated_tone'])
    # Get label distribution
    for idx, annotated_title in df.groupby('document_id'):
        label_dict = {'document_id': idx, 
                    'text': annotated_title['text'].values[0]}

        # Get number of annotators for document
        num_annotators = len(set(annotated_title['annotator']))

        # Get number of votes for document
        votes = annotated_title[['label', 'annotator']].drop_duplicates()
        vote_count = votes['label'].value_counts() / num_annotators
        
        # Add rel.label frequency to df
        for label in all_labels:
            if label in vote_count.keys():

                label_dict[label] = np.round(vote_count[label], 2)
            
            else:
                label_dict[label] = 0.00
        
        aggregated_labels = aggregated_labels.append(label_dict, ignore_index=True)

    # Get aggregated labels
    # TODO - vidjeti kako rješavati slučajeve koji jednak broj glasova za različite labele (trenutno ide po defaultu - neg -> neut -> poz)
    aggregated_labels['aggregated_sentiment'] = aggregated_labels[sentiment_labels].idxmax(axis=1).values
    aggregated_labels['aggregated_tone'] = aggregated_labels[tone_labels].idxmax(axis=1).values

    return aggregated_labels
    

In [67]:
# Set labels
sentiment_labels = ['Sentiment - NEG', 'Sentiment - NEUTR', 'Sentiment - POZ']
tone_labels = ['Ton - NEG', 'Ton - NEUTR', 'Ton - POZ']
all_labels = ['Sentiment - NEG', 'Sentiment - NEUTR', 'Sentiment - POZ', 'Ton - NEG', 'Ton - NEUTR', 'Ton - POZ' ]

# Get gold-labels
aggregated_labels = get_label_distribution(labeled_df, all_labels, sentiment_labels, tone_labels)
aggregated_labels.rename(columns={'text':'ner_tag_text'}, inplace=True)

In [68]:
aggregated_labels

,document_id,ner_tag_text,Sentiment - NEG,Sentiment - NEUTR,Sentiment - POZ,aggregated_sentiment,Ton - NEG,Ton - NEUTR,Ton - POZ,aggregated_tone
0,65,"<strong>Bandić</strong> favorit, a u drugi kru...",0.00,0.17,0.83,Sentiment - POZ,0.00,0.50,0.50,Ton - NEUTR
1,126,<strong>Emir</strong> se šokira kad sazna za F...,0.00,0.67,0.33,Sentiment - NEUTR,1.00,0.00,0.00,Ton - NEG
2,180,<strong>Boo</strong> dobio konkurenciju! Da li...,0.00,0.17,0.83,Sentiment - POZ,0.00,0.00,1.00,Ton - POZ
3,187,Ronaldo: Kladio bih se da će <strong>Leo Messi...,0.00,0.00,1.00,Sentiment - POZ,0.00,0.33,0.67,Ton - POZ
4,308,<strong>Brill</strong>: Mobiteli su stalno uz ...,0.17,0.83,0.00,Sentiment - NEUTR,0.17,0.83,0.00,Ton - NEUTR
...,...,...,...,...,...,...,...,...,...,...
2875,139071,Širimo <strong>Muzej krapinskih neandertalaca<...,0.00,0.50,0.50,Sentiment - NEUTR,0.00,0.00,1.00,Ton - POZ
2876,139162,<strong>Tomislav Tolušić</strong> izašao iz is...,0.33,0.67,0.00,Sentiment - NEUTR,0.33,0.67,0.00,Ton - NEUTR
2877,139202,"Pale cijene nafte, trgovci strahuju od širenja...",0.50,0.50,0.00,Sentiment - NEG,1.00,0.00,0.00,Ton - NEG
2878,139350,Šef <strong>Odjela za sudsku medicinu KBC-a Sp...,0.83,0.17,0.00,Sentiment - NEG,1.00,0.00,0.00,Ton - NEG


### Get documents with ambiguous labels

#### Sentiment

In [69]:
aggregated_labels.loc[((aggregated_labels['Sentiment - POZ'] >= 0.4) & (aggregated_labels['Sentiment - POZ'] <= 0.5)) | 
                      ((aggregated_labels['Sentiment - NEUTR'] >= 0.4) & (aggregated_labels['Sentiment - NEUTR'] <= 0.5))| 
                      ((aggregated_labels['Sentiment - NEG'] >= 0.4) & (aggregated_labels['Sentiment - NEG'] <= 0.5))]

,document_id,ner_tag_text,Sentiment - NEG,Sentiment - NEUTR,Sentiment - POZ,aggregated_sentiment,Ton - NEG,Ton - NEUTR,Ton - POZ,aggregated_tone
12,584,<strong>Google</strong> traži desetke stručnja...,0.00,0.5,0.50,Sentiment - NEUTR,0.00,0.33,0.67,Ton - POZ
16,647,<strong>Justin Bieber</strong> tetovirao križ ...,0.50,0.5,0.00,Sentiment - NEG,0.50,0.50,0.00,Ton - NEG
17,657,<strong>Saša Matić</strong> u Lisinskom: Slavi...,0.00,0.5,0.50,Sentiment - NEUTR,0.00,0.50,0.50,Ton - NEUTR
24,931,"Nakon Kodiaqa, stiže manji Karoq - <strong>Ško...",0.00,0.5,0.50,Sentiment - NEUTR,0.00,0.50,0.50,Ton - NEUTR
26,1027,Tajni plan Todorića: Idem na izbore s <strong>...,0.17,0.5,0.33,Sentiment - NEUTR,0.50,0.50,0.00,Ton - NEG
...,...,...,...,...,...,...,...,...,...,...
2859,138096,"Službeni Beograd prosvjedovao zbog ""nedoličnog...",0.50,0.5,0.00,Sentiment - NEG,0.83,0.00,0.17,Ton - NEG
2865,138466,<strong>Bayern</strong> ostao bez jednog od kl...,0.33,0.5,0.17,Sentiment - NEUTR,0.83,0.17,0.00,Ton - NEG
2870,138806,Rusija objavila: Preuzeli smo kontrolu nad čel...,0.00,0.5,0.50,Sentiment - NEUTR,0.67,0.33,0.00,Ton - NEG
2875,139071,Širimo <strong>Muzej krapinskih neandertalaca<...,0.00,0.5,0.50,Sentiment - NEUTR,0.00,0.00,1.00,Ton - POZ


#### Tone

In [70]:
aggregated_labels.loc[((aggregated_labels['Ton - POZ'] >= 0.4) & (aggregated_labels['Ton - POZ'] <= 0.5)) | 
                      ((aggregated_labels['Ton - NEUTR'] >= 0.4) & (aggregated_labels['Ton - NEUTR'] <= 0.5))| 
                      ((aggregated_labels['Ton - NEG'] >= 0.4) & (aggregated_labels['Ton - NEG'] <= 0.5))]

,document_id,ner_tag_text,Sentiment - NEG,Sentiment - NEUTR,Sentiment - POZ,aggregated_sentiment,Ton - NEG,Ton - NEUTR,Ton - POZ,aggregated_tone
0,65,"<strong>Bandić</strong> favorit, a u drugi kru...",0.00,0.17,0.83,Sentiment - POZ,0.00,0.50,0.5,Ton - NEUTR
13,599,Što su nosile Anna Dello Russo i ostale fashio...,0.00,0.67,0.33,Sentiment - NEUTR,0.00,0.50,0.5,Ton - NEUTR
16,647,<strong>Justin Bieber</strong> tetovirao križ ...,0.50,0.50,0.00,Sentiment - NEG,0.50,0.50,0.0,Ton - NEG
17,657,<strong>Saša Matić</strong> u Lisinskom: Slavi...,0.00,0.50,0.50,Sentiment - NEUTR,0.00,0.50,0.5,Ton - NEUTR
20,830,"<strong>Kovačić</strong>: Nema dogovora o Ini,...",0.00,1.00,0.00,Sentiment - NEUTR,0.50,0.50,0.0,Ton - NEG
...,...,...,...,...,...,...,...,...,...,...
2847,137666,Povodom <strong>Svjetskog dana kave</strong> p...,0.00,0.83,0.17,Sentiment - NEUTR,0.00,0.50,0.5,Ton - NEUTR
2848,137683,"Danijela na Instagramu prati Hanu Huljić, ali ...",0.83,0.17,0.00,Sentiment - NEG,0.50,0.50,0.0,Ton - NEG
2854,137874,<strong>HZJZ</strong> objavio znanstvena istra...,0.00,0.67,0.33,Sentiment - NEUTR,0.17,0.33,0.5,Ton - POZ
2857,137972,VIDEO Betlehemsko svjetlo mira stiže u <strong...,0.00,0.50,0.50,Sentiment - NEUTR,0.00,0.50,0.5,Ton - NEUTR


## Append document metadata

In [71]:
# Import original dataset that contains headline metadata (ner, ner_type, ner_begin, ner_end)
sentiment_anno_df = pd.read_csv('data/ner_headlines/sampled_headlines_ner.csv')
sentiment_anno_df.drop(columns=['text', 'article_id', 'title_id'], inplace=True)
sentiment_anno_df

,document_id,title,ner,ner_begin,ner_end,ner_type,portal,date_published,body,url
0,39899,Glavni pregovarač EU-a za Bexit: Britanci sada...,Bexit,26,31,MISC,index.hr,2019-04-02,Glavni pregovarač EU-a za Bexit: Britanci sada...,https://index.hr/vijesti/clanak/glavni-pregova...
1,116374,Policija podnijela prijavu protiv direktora KK...,KK Zadar,44,52,ORG,tportal.hr,2021-06-08,finale u košarci\nPolicija podnijela prijavu p...,https://tportal.hr/vijesti/clanak/policija-pod...
2,25027,FBI uhitio bivšu djevojku i suradnicu Jeffreyj...,FBI,0,3,ORG,hr.n1info.com,2015-08-31,"Uhićena je Ghislaine Maxwell, bivša djevojka i...",https://hr.n1info.com/showbiz/a523112-FBI-uhit...
3,113990,Mađarska ponovno produžila izvanredno stanje; ...,Mađarska,0,8,LOC,tportal.hr,2021-02-22,za još 90 dana\nMađarska ponovno produžila izv...,https://tportal.hr/vijesti/clanak/madarska-pon...
4,80953,"Pobjeda Borussije za vrh, Liverpool ‘peticom‘ ...",Borussije,8,17,ORG,slobodnadalmacija.hr,2021-10-16,Nogometaši Borussije preuzela je vrh Bundeslig...,https://slobodnadalmacija.hr/sport/strani-nogo...
...,...,...,...,...,...,...,...,...,...,...
2995,110519,Masovna uhićenja na prosvjedima protiv Lukašen...,Lukašenka,39,48,PER,tportal.hr,2020-09-06,VODENI TOPOVI I OKLOPNJACI\nMasovna uhićenja n...,https://tportal.hr/vijesti/clanak/masovna-uhic...
2996,109989,Nakon 18 godina su se vratili u Premier ligu i...,Premier ligu,32,44,ORG,tportal.hr,2020-08-07,hrvat na meti velikana\nNakon 18 godina su se ...,https://trazilica.tportal.hr/sport/clanak/nako...
2997,20017,Macron otkrio zašto ne želi koristiti termin “...,Ukrajini,74,82,LOC,face.ba,2022-04-14,Macron otkrio zašto ne želi koristiti termin “...,https://www.face.ba/vijesti/rat-u-ukrajini/mac...
2998,98259,Preminuo dugogodišnji glumac kazališta 'Komedi...,Komedija,40,48,ORG,tportal.hr,2018-09-02,TUŽNA VIJEST\nPreminuo dugogodišnji glumac kaz...,https://www.tportal.hr/kultura/clanak/preminuo...


In [72]:
# Merge labeld and original dataset
aggregated_labels = pd.merge(aggregated_labels, sentiment_anno_df, on='document_id')
aggregated_labels.rename(columns={'title':'text'}, inplace=True)

In [73]:
aggregated_labels

,document_id,ner_tag_text,Sentiment - NEG,Sentiment - NEUTR,Sentiment - POZ,aggregated_sentiment,Ton - NEG,Ton - NEUTR,Ton - POZ,aggregated_tone,text,ner,ner_begin,ner_end,ner_type,portal,date_published,body,url
0,65,"<strong>Bandić</strong> favorit, a u drugi kru...",0.00,0.17,0.83,Sentiment - POZ,0.00,0.50,0.50,Ton - NEUTR,"Bandić favorit, a u drugi krug bi mogao ići i ...",Bandić,0,6,PER,24sata.hr,2013-03-28,"Bandić favorit, a u drugi krug bi mogao ići i ...",https://www.24sata.hr/news/bandic-favorit-a-u-...
1,126,<strong>Emir</strong> se šokira kad sazna za F...,0.00,0.67,0.33,Sentiment - NEUTR,1.00,0.00,0.00,Ton - NEG,"Emir se šokira kad sazna za Ferihinu laž, Gund...",Emir,0,4,PER,24sata.hr,2013-07-24,"Emir se šokira kad sazna za Ferihinu laž, Gund...",https://www.24sata.hr/show/emir-se-sokira-kad-...
2,180,<strong>Boo</strong> dobio konkurenciju! Da li...,0.00,0.17,0.83,Sentiment - POZ,0.00,0.00,1.00,Ton - POZ,Boo dobio konkurenciju! Da li je ovo najslađi ...,Boo,0,3,PER,24sata.hr,2013-11-21,Boo dobio konkurenciju! Da li je ovo najslađi ...,https://www.24sata.hr/ljubimci/boo-dobio-konku...
3,187,Ronaldo: Kladio bih se da će <strong>Leo Messi...,0.00,0.00,1.00,Sentiment - POZ,0.00,0.33,0.67,Ton - POZ,Ronaldo: Kladio bih se da će Leo Messi osvojit...,Leo Messi,29,38,PER,24sata.hr,2013-12-13,Ronaldo: Kladio bih se da će Leo Messi osvojit...,https://www.24sata.hr/sport/ronaldo-kladio-bih...
4,308,<strong>Brill</strong>: Mobiteli su stalno uz ...,0.17,0.83,0.00,Sentiment - NEUTR,0.17,0.83,0.00,Ton - NEUTR,"Brill: Mobiteli su stalno uz nas, tako bi i va...",Brill,0,5,PER,24sata.hr,2014-11-21,"Brill: Mobiteli su stalno uz nas, tako bi i va...",https://www.24sata.hr/tech/brill-mobiteli-su-s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,139071,Širimo <strong>Muzej krapinskih neandertalaca<...,0.00,0.50,0.50,Sentiment - NEUTR,0.00,0.00,1.00,Ton - POZ,"Širimo Muzej krapinskih neandertalaca, a u dvo...",Muzej krapinskih neandertalaca,7,37,ORG,vecernji.hr,2022-06-10,Početkom 2021. godine na čelo Muzeja Hrvatskog...,https://vecernji.hr/kultura/sirimo-muzej-krapi...
2876,139162,<strong>Tomislav Tolušić</strong> izašao iz is...,0.33,0.67,0.00,Sentiment - NEUTR,0.33,0.67,0.00,Ton - NEUTR,Tomislav Tolušić izašao iz istražnog zatvora,Tomislav Tolušić,0,16,PER,vecernji.hr,2022-07-01,Tomislav Tolušić izašao je danas iz istražnog ...,https://www.vecernji.hr/vijesti/tomislav-tolus...
2877,139202,"Pale cijene nafte, trgovci strahuju od širenja...",0.50,0.50,0.00,Sentiment - NEG,1.00,0.00,0.00,Ton - NEG,"Pale cijene nafte, trgovci strahuju od širenja...",Kini,62,66,LOC,vecernji.hr,2022-07-11,Trgovci su i dalje nervozni zbog planova zapad...,https://m.vecernji.hr/biznis/pale-cijene-nafte...
2878,139350,Šef <strong>Odjela za sudsku medicinu KBC-a Sp...,0.83,0.17,0.00,Sentiment - NEG,1.00,0.00,0.00,Ton - NEG,Šef Odjela za sudsku medicinu KBC-a Split: 'Šo...,Odjela za sudsku medicinu KBC - a Split,4,41,ORG,vecernji.hr,2022-08-08,"Alujević ističe da je ""patologinja koja je žur...",https://m.vecernji.hr/vijesti/sef-odjela-za-su...


### Remove multiple whitespaces around NE that contain '-' 

In [74]:
# TODO - napisati ovo malo pametnije
# Bug u 'ner' stupcu jer ponekad ner string ne odgovara formatom ner stringu u naslovu 
aggregated_labels['ner'] = aggregated_labels['ner'].apply(lambda x: x.replace(' - ', '-'))
aggregated_labels['ner'] = aggregated_labels['ner'].apply(lambda x: x.replace(' -', '-'))
aggregated_labels['ner'] = aggregated_labels['ner'].apply(lambda x: x.replace(' / ', '/'))
aggregated_labels['ner'] = aggregated_labels['ner'].apply(lambda x: x.replace('Val d ` Isere', 'Val d`Isere'))
aggregated_labels['ner'] = aggregated_labels['ner'].apply(lambda x: x.replace('Renault Twingo R. S', 'Renault Twingo R.S'))
aggregated_labels['ner'] = aggregated_labels['ner'].apply(lambda x: x.replace("Zračnoj luci'Franjo Tuđman", "Zračnoj luci 'Franjo Tuđman"))
aggregated_labels['ner'] = aggregated_labels['ner'].apply(lambda x: x.replace("Croatia Open-Dancestar 2010", "Croatia Open- Dancestar 2010"))
aggregated_labels['ner'] = aggregated_labels['ner'].apply(lambda x: x.replace("J. Lo", "J.Lo"))
aggregated_labels['ner'] = aggregated_labels['ner'].apply(lambda x: x.replace("Teatru & TD", "Teatru &TD"))
aggregated_labels['ner'] = aggregated_labels['ner'].apply(lambda x: x.replace("J & T", "J&T"))

## Get target order

In [75]:
def get_target_order(df):

    target_orders = []

    for idx, row in df.iterrows():

        begin_ner = row['ner_begin']
        target_entity = row['ner']
        text = row['text']

        # Get target id (= target occurrence)
        ids = np.array([m.start() for m in re.finditer(target_entity, text)])
        target_order = np.where(ids == begin_ner)[0][0]
        
        df.loc[idx, ['target_order']] = target_order
    
    df['target_order'] = df['target_order'].astype('int')

    return df

In [76]:
aggregated_labels = get_target_order(aggregated_labels)
aggregated_labels

,document_id,ner_tag_text,Sentiment - NEG,Sentiment - NEUTR,Sentiment - POZ,aggregated_sentiment,Ton - NEG,Ton - NEUTR,Ton - POZ,aggregated_tone,text,ner,ner_begin,ner_end,ner_type,portal,date_published,body,url,target_order
0,65,"<strong>Bandić</strong> favorit, a u drugi kru...",0.00,0.17,0.83,Sentiment - POZ,0.00,0.50,0.50,Ton - NEUTR,"Bandić favorit, a u drugi krug bi mogao ići i ...",Bandić,0,6,PER,24sata.hr,2013-03-28,"Bandić favorit, a u drugi krug bi mogao ići i ...",https://www.24sata.hr/news/bandic-favorit-a-u-...,0
1,126,<strong>Emir</strong> se šokira kad sazna za F...,0.00,0.67,0.33,Sentiment - NEUTR,1.00,0.00,0.00,Ton - NEG,"Emir se šokira kad sazna za Ferihinu laž, Gund...",Emir,0,4,PER,24sata.hr,2013-07-24,"Emir se šokira kad sazna za Ferihinu laž, Gund...",https://www.24sata.hr/show/emir-se-sokira-kad-...,0
2,180,<strong>Boo</strong> dobio konkurenciju! Da li...,0.00,0.17,0.83,Sentiment - POZ,0.00,0.00,1.00,Ton - POZ,Boo dobio konkurenciju! Da li je ovo najslađi ...,Boo,0,3,PER,24sata.hr,2013-11-21,Boo dobio konkurenciju! Da li je ovo najslađi ...,https://www.24sata.hr/ljubimci/boo-dobio-konku...,0
3,187,Ronaldo: Kladio bih se da će <strong>Leo Messi...,0.00,0.00,1.00,Sentiment - POZ,0.00,0.33,0.67,Ton - POZ,Ronaldo: Kladio bih se da će Leo Messi osvojit...,Leo Messi,29,38,PER,24sata.hr,2013-12-13,Ronaldo: Kladio bih se da će Leo Messi osvojit...,https://www.24sata.hr/sport/ronaldo-kladio-bih...,0
4,308,<strong>Brill</strong>: Mobiteli su stalno uz ...,0.17,0.83,0.00,Sentiment - NEUTR,0.17,0.83,0.00,Ton - NEUTR,"Brill: Mobiteli su stalno uz nas, tako bi i va...",Brill,0,5,PER,24sata.hr,2014-11-21,"Brill: Mobiteli su stalno uz nas, tako bi i va...",https://www.24sata.hr/tech/brill-mobiteli-su-s...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,139071,Širimo <strong>Muzej krapinskih neandertalaca<...,0.00,0.50,0.50,Sentiment - NEUTR,0.00,0.00,1.00,Ton - POZ,"Širimo Muzej krapinskih neandertalaca, a u dvo...",Muzej krapinskih neandertalaca,7,37,ORG,vecernji.hr,2022-06-10,Početkom 2021. godine na čelo Muzeja Hrvatskog...,https://vecernji.hr/kultura/sirimo-muzej-krapi...,0
2876,139162,<strong>Tomislav Tolušić</strong> izašao iz is...,0.33,0.67,0.00,Sentiment - NEUTR,0.33,0.67,0.00,Ton - NEUTR,Tomislav Tolušić izašao iz istražnog zatvora,Tomislav Tolušić,0,16,PER,vecernji.hr,2022-07-01,Tomislav Tolušić izašao je danas iz istražnog ...,https://www.vecernji.hr/vijesti/tomislav-tolus...,0
2877,139202,"Pale cijene nafte, trgovci strahuju od širenja...",0.50,0.50,0.00,Sentiment - NEG,1.00,0.00,0.00,Ton - NEG,"Pale cijene nafte, trgovci strahuju od širenja...",Kini,62,66,LOC,vecernji.hr,2022-07-11,Trgovci su i dalje nervozni zbog planova zapad...,https://m.vecernji.hr/biznis/pale-cijene-nafte...,0
2878,139350,Šef <strong>Odjela za sudsku medicinu KBC-a Sp...,0.83,0.17,0.00,Sentiment - NEG,1.00,0.00,0.00,Ton - NEG,Šef Odjela za sudsku medicinu KBC-a Split: 'Šo...,Odjela za sudsku medicinu KBC-a Split,4,41,ORG,vecernji.hr,2022-08-08,"Alujević ističe da je ""patologinja koja je žur...",https://m.vecernji.hr/vijesti/sef-odjela-za-su...,0


## Save to csv

In [77]:
# Rearrange columns
aggregated_labels = aggregated_labels[['document_id', 'text', 'ner_tag_text', 'ner', 'ner_type', 'ner_begin', 'ner_end', 'target_order', 'Sentiment - NEG', 'Sentiment - NEUTR',
                   'Sentiment - POZ', 'aggregated_sentiment', 'Ton - NEG', 'Ton - NEUTR',
                   'Ton - POZ', 'aggregated_tone'
                  ]]

aggregated_labels

,document_id,text,ner_tag_text,ner,ner_type,ner_begin,ner_end,target_order,Sentiment - NEG,Sentiment - NEUTR,Sentiment - POZ,aggregated_sentiment,Ton - NEG,Ton - NEUTR,Ton - POZ,aggregated_tone
0,65,"Bandić favorit, a u drugi krug bi mogao ići i ...","<strong>Bandić</strong> favorit, a u drugi kru...",Bandić,PER,0,6,0,0.00,0.17,0.83,Sentiment - POZ,0.00,0.50,0.50,Ton - NEUTR
1,126,"Emir se šokira kad sazna za Ferihinu laž, Gund...",<strong>Emir</strong> se šokira kad sazna za F...,Emir,PER,0,4,0,0.00,0.67,0.33,Sentiment - NEUTR,1.00,0.00,0.00,Ton - NEG
2,180,Boo dobio konkurenciju! Da li je ovo najslađi ...,<strong>Boo</strong> dobio konkurenciju! Da li...,Boo,PER,0,3,0,0.00,0.17,0.83,Sentiment - POZ,0.00,0.00,1.00,Ton - POZ
3,187,Ronaldo: Kladio bih se da će Leo Messi osvojit...,Ronaldo: Kladio bih se da će <strong>Leo Messi...,Leo Messi,PER,29,38,0,0.00,0.00,1.00,Sentiment - POZ,0.00,0.33,0.67,Ton - POZ
4,308,"Brill: Mobiteli su stalno uz nas, tako bi i va...",<strong>Brill</strong>: Mobiteli su stalno uz ...,Brill,PER,0,5,0,0.17,0.83,0.00,Sentiment - NEUTR,0.17,0.83,0.00,Ton - NEUTR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,139071,"Širimo Muzej krapinskih neandertalaca, a u dvo...",Širimo <strong>Muzej krapinskih neandertalaca<...,Muzej krapinskih neandertalaca,ORG,7,37,0,0.00,0.50,0.50,Sentiment - NEUTR,0.00,0.00,1.00,Ton - POZ
2876,139162,Tomislav Tolušić izašao iz istražnog zatvora,<strong>Tomislav Tolušić</strong> izašao iz is...,Tomislav Tolušić,PER,0,16,0,0.33,0.67,0.00,Sentiment - NEUTR,0.33,0.67,0.00,Ton - NEUTR
2877,139202,"Pale cijene nafte, trgovci strahuju od širenja...","Pale cijene nafte, trgovci strahuju od širenja...",Kini,LOC,62,66,0,0.50,0.50,0.00,Sentiment - NEG,1.00,0.00,0.00,Ton - NEG
2878,139350,Šef Odjela za sudsku medicinu KBC-a Split: 'Šo...,Šef <strong>Odjela za sudsku medicinu KBC-a Sp...,Odjela za sudsku medicinu KBC-a Split,ORG,4,41,0,0.83,0.17,0.00,Sentiment - NEG,1.00,0.00,0.00,Ton - NEG


In [56]:
# Contains 2880 annotated headlines (only error titles are omitted)
aggregated_labels.to_csv('data/gold_label/stone_3k.csv', index=False)